In [9]:
import json
import numpy as np
import collections
import pandas as pd
import csv
import random

In [13]:
filename = "baseline_logs_01.json"

In [14]:
with open(filename, 'r') as f:
    all_logs = json.load(f)

In [4]:
all_logs[1294]

{'insertId': '1d41hkig1097li2',
 'jsonPayload': {'created': 1532326203.583701,
  'levelname': 'INFO',
  'lineno': 326,
  'message': '[batch 137]: seen 5299200 words at 3202.4 wps, loss = 2.428',
  'pathname': '/runcloudml.py'},
 'labels': {'compute.googleapis.com/resource_id': '2080932681024947335',
  'compute.googleapis.com/resource_name': 'cmle-training-ps-9780a94e1c-3-lv4rr',
  'compute.googleapis.com/zone': 'us-central1-c',
  'ml.googleapis.com/job_id': 'baseline_test_complex_model_l_03',
  'ml.googleapis.com/job_id/log_area': 'root',
  'ml.googleapis.com/task_name': 'ps-replica-3',
  'ml.googleapis.com/trial_id': ''},
 'logName': 'projects/muthderd-w266-000001/logs/ps-replica-3',
 'receiveTimestamp': '2018-07-23T06:10:08.919353180Z',
 'resource': {'labels': {'job_id': 'baseline_test_complex_model_l_03',
   'project_id': 'muthderd-w266-000001',
   'task_name': 'ps-replica-3'},
  'type': 'ml_job'},
 'severity': 'INFO',
 'timestamp': '2018-07-23T06:10:03.583700895Z'}

In [5]:
all_logs[1294]['jsonPayload']['message']

'[batch 137]: seen 5299200 words at 3202.4 wps, loss = 2.428'

In [6]:
type(all_logs[0])

dict

In [7]:
all_logs[0]['textPayload']

'Job cancelled.'

In [4]:
generated_reviews = []
for log in all_logs:
    if 'jsonPayload' in log.keys():
        payload = log['jsonPayload']
        if 'message' in payload.keys():
            message = payload['message']
            if "SOR" in message:
                generated_reviews.append(message)
    #message = log['textPayload']
    #if "SOR" in message:
        #generated_reviews.append(message)

In [5]:
for review in generated_reviews:
    print(review)

<SOR>of your browses, at least a few years later this is not to look thing, higher than the other folks in the clothing's, thats ordering this cook for. th
<SOR>huge pool and cheese. the office is really gining our trip and driving up away, as well.  special artist was moving and few much in my night for a few
<SOR>buffet in the day.  i will be back!<EOR>
<SOR>went in, and loved it!!!<EOR>
<SOR>food!<EOR>
<SOR>one more than i stayed!<EOR>
<SOR>the waitress sandwiched presentations on the chicken price conversation. their milk showers and sweetness energy worth you roof including & over rich 
<SOR>is the new york.spicy, a creamy potatoes and osh salad book needs to eat it, unfortunately stobe bels 's.i noticed that shenry is from best part of th
<SOR>better combinations, friendly events. there are bonus!!!!i've seen the routine thing to suggest you have two stretches at a small card and my goodness
<SOR>my taco is normally!! the best and this place is actually stalking and after we've h

In [24]:
generated_reviews[0]

'<SOR>(honest for extensive each class looking for the layer mushroom store) was the sample of peanuts in the shape of country. creamy, pulled pork, atmosph'

In [19]:
#build a list of list of characters from the 5-star reviews
def preprocess_review_series(review_series):
    review_list = []
    for new_review in review_series:
        clipped_review = new_review[2:-1]
        char_list = list(clipped_review.lower())
        semifinal_review = []
        last_char = ''
        for ascii_char in char_list:
            if ascii_char == '\\' or last_char == '\\':
                pass
            else:
                #isascii = lambda s: len(s) == len(s.encode())
                semifinal_review.append(ascii_char)
            last_char = ascii_char
        if len(semifinal_review) > 300:
            final_review = ['<SOR>'] + semifinal_review + ['<EOR>']
            #print(final_review)
            review_list.append(final_review)
    return review_list

def get_review_series(review_path = '/home/kalvin_kao/yelp_challenge_dataset/review.csv'):
    #review_path = '/home/kalvin_kao/yelp_challenge_dataset/review.csv'
    review_df = pd.read_csv(review_path)
    five_star_review_df = review_df[review_df['stars']==5]
    #five_star_review_series = five_star_review_df['text']
    return five_star_review_df['text']

def get_business_list(business_path = '/home/kalvin_kao/yelp_challenge_dataset/business.csv'):
    #business_path = '/home/kalvin_kao/yelp_challenge_dataset/business.csv'
    return pd.read_csv(business_path)

def split_train_test(review_list, training_samples, test_samples):
    #pass in randomized review list
    train_len = int(np.floor(0.8*len(review_list)))
    test_len = int(np.floor(0.2*len(review_list)))
    training_review_list = review_list[:train_len]
    testing_review_list = review_list[-test_len:]
    randomized_training_list = random.sample(training_review_list, training_samples)
    randomized_testing_list = random.sample(testing_review_list, test_samples)
    training_review_list = [item for sublist in randomized_training_list for item in sublist]
    print("number of training characters", len(training_review_list))
    test_review_list = [item for sublist in randomized_testing_list for item in sublist]
    print("number of test characters", len(test_review_list))
    return randomized_training_list, randomized_testing_list

def make_train_test_data(five_star_review_series, training_samples=25000, test_samples=6250):
    #fix randomization to prevent evaluation on trained samples
    review_list = preprocess_review_series(five_star_review_series)
    np.random.shuffle(review_list)
    #split into 3 for 1) attack, 2) defense, and 3) GAN
    one_third_len = int(np.floor(0.33*len(review_list)))
    attack_review_list = review_list[:one_third_len]
    #attack_training_samples = 25000
    #attack_test_samples = 6250
    defense_review_list = review_list[one_third_len:(2*one_third_len)]
    #defense_training_samples = 25000
    #defense_test_samples = 6250
    gan_review_list = review_list[-one_third_len:]
    #gan_training_samples = 25000
    #gan_test_samples = 6250
    #split and shuffle the data for attack
    attack_training_review_list, attack_test_review_list = split_train_test(attack_review_list, training_samples, test_samples)
    defense_training_review_list, defense_test_review_list = split_train_test(attack_review_list, training_samples, test_samples)
    gan_training_review_list, gan_test_review_list = split_train_test(attack_review_list, training_samples, test_samples)
    return attack_training_review_list, attack_test_review_list, defense_training_review_list, defense_test_review_list, gan_training_review_list, gan_test_review_list

def make_vocabulary(training_review_list, test_review_list):
    unique_characters = list(set(training_review_list + test_review_list))
    #vocabulary
    char_dict = {w:i for i, w in enumerate(unique_characters)}
    ids_to_words = {v: k for k, v in char_dict.items()}
    return char_dict, ids_to_words

def convert_to_ids(char_dict, review_list):
    #convert to flat (1D) np.array(int) of ids
    review_ids = [char_dict.get(token) for token in review_list]
    return np.array(review_ids)

In [7]:
review_path = '/home/kalvin_kao/yelp_challenge_dataset/review.csv'

In [22]:
five_star_reviews = get_review_series(review_path)
training_samples = 25000
test_samples = 6250
attack_training_review_list, attack_test_review_list, defense_training_review_list, defense_test_review_list, gan_training_review_list, gan_test_review_list = make_train_test_data(five_star_reviews, training_samples, test_samples)

number of training characters 17890192
number of test characters 4468583
number of training characters 18091570
number of test characters 4481422
number of training characters 17864112
number of test characters 4496107


In [20]:
np.random.shuffle(generated_reviews)
generated_training_review_list, generated_test_review_list = split_train_test(generated_reviews, training_samples=20, test_samples=5)

number of training characters 2979
number of test characters 733


In [25]:
generated_training_review_list[0]

"<SOR>a true hard fix!!! the owner, egg, olive and potato was classy, simple and ill take the time to even have found the food is damn again i've never been"

In [21]:
with open("artificial_train_data.csv", "w") as output:
    writer = csv.writer(output, lineterminator='\n')
    writer.writerows(generated_training_review_list)

In [ ]:
with open("artificial_test_data.csv", "w") as output:
    writer = csv.writer(output, lineterminator='\n')
    writer.writerows(generated_test_review_list)

In [23]:
with open("attack_train_data.csv", "w") as output:
    writer = csv.writer(output, lineterminator='\n')
    writer.writerows(attack_training_review_list)

In [ ]:
with open("attack_test_data.csv", "w") as output:
    writer = csv.writer(output, lineterminator='\n')
    writer.writerows(attack_test_review_list)

In [ ]:
with open("defense_train_data.csv", "w") as output:
    writer = csv.writer(output, lineterminator='\n')
    writer.writerows(defense_training_review_list)

In [ ]:
with open("defense_test_data.csv", "w") as output:
    writer = csv.writer(output, lineterminator='\n')
    writer.writerows(defense_test_review_list)

In [ ]:
with open("gan_train_data.csv", "w") as output:
    writer = csv.writer(output, lineterminator='\n')
    writer.writerows(gan_training_review_list)

In [ ]:
with open("gan_test_data.csv", "w") as output:
    writer = csv.writer(output, lineterminator='\n')
    writer.writerows(gan_test_review_list)

In [39]:
with open("attack_train_data.csv", 'r') as csvfile:
    counter = 0
    reader = csv.reader(csvfile, delimiter=',')
    for row in reader:
        print(row)
        print()
        if counter > 10:
            break
        counter += 1
    new_training_review_list = [item for sublist in reader for item in sublist]

['<SOR>', 'b', 'e', 's', 't', ' ', 'c', 'l', 'u', 'b', ' ', 'i', "'", 'v', 'e', ' ', 'b', 'e', 'e', 'n', ' ', 't', 'o', ' ', 'i', 'n', ' ', 'v', 'e', 'g', 'a', 's', '-', '-', '-', 'p', 'r', 'o', 'b', 'a', 'b', 'l', 'y', ' ', 't', 'h', 'e', ' ', 'b', 'e', 's', 't', ' ', 'i', "'", 'v', 'e', ' ', 'b', 'e', 'e', 'n', ' ', 't', 'o', ' ', 'e', 'v', 'e', 'r', '.', 'h', 'i', 'p', ' ', 'h', 'o', 'p', ' ', 'r', 'o', 'o', 'm', ' ', 'i', 's', ' ', 's', 'm', 'a', 'l', 'l', ' ', 'b', 'u', 't', ' ', 't', 'h', 'e', 'y', ' ', 'p', 'l', 'a', 'y', ' ', 's', 'o', 'm', 'e', ' ', 'g', 'o', 'o', 'd', ' ', 'o', 'l', 'd', 'i', 'e', 's', ' ', '(', 'r', 'a', 'p', '/', 'h', 'i', 'p', 'h', 'o', 'p', ')', ' ', 't', 'h', 'a', 't', ' ', 'i', ' ', 'n', 'e', 'v', 'e', 'r', ' ', 'k', 'n', 'e', 'w', ' ', 'i', ' ', 'm', 'i', 's', 's', 'e', 'd', ' ', 's', 'o', ' ', 'm', 'u', 'c', 'h', '.', 't', 'h', 'e', 'i', 'r', ' ', 'l', 'i', 'g', 'h', 't', ' ', 'i', 'n', 's', 't', 'a', 'l', 'l', 'a', 't', 'i', 'o', 'n', ' ', 'i', 'n', 

In [42]:
new_training_review_list[100:150]

['(',
 'a',
 'r',
 'o',
 'u',
 'n',
 'd',
 ' ',
 '2',
 ' ',
 'h',
 'o',
 'u',
 'r',
 's',
 ')',
 ' ',
 'd',
 'u',
 'r',
 'i',
 'n',
 'g',
 ' ',
 'd',
 'i',
 'n',
 'n',
 'e',
 'r',
 ' ',
 't',
 'i',
 'm',
 'e',
 ',',
 ' ',
 'b',
 'u',
 't',
 ' ',
 'y',
 'o',
 'u',
 ' ',
 'c',
 'a',
 'n',
 ' ',
 'w']

In [ ]:
words_to_ids, ids_to_words = make_vocabulary(train_review_list, test_review_list)
train_ids = convert_to_ids(words_to_ids, train_review_list)
test_ids = convert_to_ids(words_to_ids, test_review_list)

In [10]:
seen = {}
dupes = []

for x in generated_reviews:
    if x not in seen:
        seen[x] = 1
    else:
        if seen[x] == 1:
            dupes.append(x)
        seen[x] += 1

In [11]:
dupes

[]

In [12]:
"<EOR>" in generated_reviews[95]

True

In [13]:
generated_reviews[95].find('<EOR>')

43

In [14]:
generated_reviews[95]

'<SOR>and all just when i wants to wait!!!!!<EOR>'

In [15]:
generated_reviews[95][:43]

'<SOR>and all just when i wants to wait!!!!!'

In [26]:
clipped_review_list = []
for review in generated_reviews:
    clipped_review = review[5:]
    if "<EOR>" in clipped_review:
        eor_index = clipped_review.find('<EOR>')
        clipped_review = clipped_review[:eor_index]
    clipped_review_list.append(clipped_review)

In [28]:
generated_review_list = []
for review in clipped_review_list:
    #print(type(review))
    char_list = list(review.lower())
    semifinal_review = []
    last_char = ''
    for ascii_char in char_list:
        if ascii_char == '\\' or last_char == '\\':
            pass
        else:
            #isascii = lambda s: len(s) == len(s.encode())
            semifinal_review.append(ascii_char)
        last_char = ascii_char
    #if len(semifinal_review) > 300:
    final_review = ['<SOR>'] + semifinal_review + ['<EOR>']
        #print(final_review)
    generated_review_list.append(final_review)

In [29]:
np.random.shuffle(generated_review_list)
generated_training_review_list, generated_test_review_list = split_train_test(generated_review_list, training_samples=20, test_samples=5)

number of training characters 2872
number of test characters 633


In [30]:
with open("artificial_train_data.csv", "w") as output:
    writer = csv.writer(output, lineterminator='\n')
    writer.writerows(generated_training_review_list)

In [37]:
with open("artificial_train_data.csv", 'r') as csvfile:
    reader = csv.reader(csvfile, delimiter=',')
    for row in reader:
        print(row)
        #print(''.join(row))
        print()

['<SOR>', 'w', 'h', 'e', 'n', ' ', 'w', 'e', ' ', 'c', 'a', 'n', "'", 't', ' ', 'f', 'i', 'n', 'd', ' ', 'b', 'u', 't', ' ', 'p', 'l', 'a', 'c', 'e', ' ', 'w', 'a', 's', ' ', 'l', 'a', 'r', 'g', 'e', ' ', 'f', 'o', 'r', ' ', 'm', 'y', ' ', 'f', 'a', 'v', 'o', 'r', 'i', 't', 'e', ' ', 'p', 'l', 'a', 'c', 'e', ' ', 'w', 'h', 'e', 'r', 'e', ' ', 'o', 'n', 'e', ' ', 'o', 'f', ' ', 't', 'h', 'o', 's', 'e', ' ', 'd', 'i', 'n', 'n', 'e', 'r', 's', ' ', 'a', 'r', 'e', ' ', 'a', 'l', 's', 'o', ' ', 'g', 'r', 'e', 'a', 't', '.', ' ', 'w', 'e', ' ', 'l', 'i', 'v', 'e', ' ', 'i', 'n', ' ', 't', 'h', 'e', ' ', 'e', 'a', 'r', 'l', 'y', ' ', 'a', 'n', 'd', ' ', 'a', 'l', 'w', 'a', 'y', 's', ' ', 'r', 'e', 't', 'u', 'r', 'n', '!', ' ', 'l', 'e', 'a', 'v', 'e', ' ', '<EOR>']

['<SOR>', 'i', 'm', 'm', 'e', 'd', 'i', 'a', 't', 'e', 'l', 'y', ' ', 'w', 'i', 't', 'h', ' ', 't', 'a', 'x', 'a', 'k', 'i', 'n', ' ', 'c', 'h', 'i', 'l', 'i', ' ', 'p', 'e', 'a', 'n', 'u', 't', 's', ',', ' ', 'c', 'h', 'a', 'n', 

In [18]:
generated_review_char_list = [item for sublist in clipped_review_list for item in sublist]

In [19]:
generated_chars = pd.Series(generated_review_char_list)

In [20]:
generated_chars.describe()

count     13070
unique       56
top            
freq       2376
dtype: object

In [21]:
#import collections
print([item for item, count in collections.Counter(generated_reviews).items() if count > 1])

[]


In [22]:
char_counts = collections.Counter(generated_review_char_list).items()

In [23]:
for item in char_counts:
    print(item)

('o', 762)
('f', 286)
(' ', 2376)
('y', 253)
('u', 288)
('r', 602)
('b', 156)
('w', 252)
('s', 646)
('e', 1294)
(',', 86)
('a', 913)
('t', 934)
('l', 430)
('h', 514)
('i', 686)
('n', 624)
('k', 97)
('g', 229)
('c', 283)
("'", 34)
('d', 400)
('.', 147)
('p', 210)
('v', 112)
('m', 235)
('!', 63)
('&', 2)
('z', 21)
('j', 20)
('q', 13)
('-', 15)
('(', 14)
(')', 7)
('"', 2)
('=', 1)
('x', 13)
(';', 1)
(':', 3)
('1', 8)
('8', 3)
('4', 1)
('5', 4)
('2', 3)
('*', 2)
('7', 2)
('+', 1)
('?', 3)
('{', 1)
('$', 3)
('#', 1)
('0', 6)
('6', 1)
('3', 4)
('9', 2)
('/', 1)


In [68]:
char_list = []
count_list = []
for a in char_counts:
    #print(a[0])
    #print(a[1])
    if a[0] != " ":
        #print(a[1])
        char_list.append(a[0])
        count_list.append(a[1])

In [ ]:
#total - spaces
total_chars = 1370 - 236
